# 코드 실행속도를 알고 싶다면 `%timeit` and `%time`을 활용하자

`% timeit`은 자동적으로 반복 횟수를 결정 합니다. 속도가 느린 명령의 경우, 더 적은 반복을 수행합니다 

In [11]:
%timeit sum(range(100))

21.4 µs ± 318 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

481 ms ± 8.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

2.04 ms ± 288 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


`%time` 매직 함수는 장기 실행 명령에 적합한 옵션일 수 있습니다.

In [14]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
Wall time: 32 ms


In [15]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
Wall time: 4 ms


``% timeit``와 같이``% time``의 경우, 이중 퍼센트 기호 매직 구문을 사용하면 다중 행 스크립트의 타이밍을 허용합니다.

In [16]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

Wall time: 589 ms


# 코드 프로파일링은 `%prun`을 활용하자

In [17]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total
%prun sum_of_lists(1000000)

```ipython
         29 function calls in 1.086 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.725    0.145    0.725    0.145 <ipython-input-10-df1d00808770>:4(<listcomp>)
        5    0.308    0.062    0.308    0.062 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.042    0.042    1.075    1.075 <ipython-input-10-df1d00808770>:1(sum_of_lists)
        1    0.011    0.011    1.086    1.086 <string>:1(<module>)
        5    0.000    0.000    0.308    0.062 fromnumeric.py:1778(sum)
        1    0.000    0.000    1.086    1.086 {built-in method builtins.exec}
        5    0.000    0.000    0.308    0.062 _methods.py:31(_sum)
        5    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
```

In [18]:
%load_ext line_profiler

한 줄씩 프로파일링을 할 경우 `% lprun`을 사용할 수 있습니다.

```ipython
pip install line-profiler
```

In [19]:
%lprun -f sum_of_lists sum_of_lists(5000)

```ipython
Timer unit: 3.0186e-07 s

Total time: 0.00917474 s
File: <ipython-input-10-df1d00808770>
Function: sum_of_lists at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def sum_of_lists(N):
     2         1         23.0     23.0      0.1      total = 0
     3         6         28.0      4.7      0.1      for i in range(5):
     4         5      24410.0   4882.0     80.3          L = [j ^ (j >> i) for j in range(N)]
     5         5       5931.0   1186.2     19.5          total += sum(L)
     6         1          2.0      2.0      0.0      return total
```

# 메모리 프로파일링은 `%memit` and `%mprun`

메모리 프로파일링을 위해선 아래 라이브러리를 설치

```ipython
$ pip install memory_profiler
```

In [20]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [21]:
%memit sum_of_lists(1000000)

peak memory: 146.52 MiB, increment: 66.02 MiB


`%mprun`는 파일을 기반으로 분석하기 때문에 파일 작성 후 프로파일링을 실행

In [31]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Writing mprun_demo.py


In [32]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)

```ipython
Filename: E:\Developer\works\Study-Pandas\python-data-science-handbook\mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     1     80.9 MiB     80.9 MiB   def sum_of_lists(N):
     2     80.9 MiB      0.0 MiB       total = 0
     3     80.9 MiB      0.0 MiB       for i in range(5):
     4    117.7 MiB -58953117.7 MiB           L = [j ^ (j >> i) for j in range(N)]
     5    117.7 MiB      0.0 MiB           total += sum(L)
     6     80.6 MiB   -156.2 MiB           del L # remove reference to L
     7     80.6 MiB     -0.4 MiB       return total
```